## Extracting unique characters from a dataset

In [1]:
#file_path = "/mnt/resource/Etc/data/adrita18/metadata.csv"
#file_path = "/mnt/resource/Etc/data/alamgir12/metadata.csv"
file_path = "/media/arif/Resource/Etc/data/alamgir12/metadata.csv"


text = ""
with open(file_path) as f:
    for i, line in enumerate(f):
        _, t, nt = line.split("|")
        text += nt.strip()

        if i < 5:
            print(nt)

characters = set(text)

print(f'Total unique characters: {len(characters)}')
print(f'Characters: {"".join(sorted(characters))}')

 স্কুলটির দিকে দৃষ্টি দিন।

 পিরোজপুর জেলার কাউখালী বন্দরে একটি মাত্র বালিকা বিদ্যালয় আছে, যা এসবি সরকারি উচ্চবালিকা বিদ্যালয় নামে পরিচিত।

 অনেক ঐতিহ্য বহন করছে এ বিদ্যালয়টি।

 কিন্তু এখানকার ছাত্রী ও শিক্ষক-শিক্ষিকাদের সম্মুখীন হতে হচ্ছে বহুবিধ সমস্যার!

 পাশ্বর্বর্তী বিভিন্ন গ্রাম ও দূরদূরান্ত থেকে ভালো শিক্ষার আশায় এখানে পড়তে আসছে শিক্ষার্থীরা।

Total unique characters: 97
Characters:  !"'(),-./:;?[] ।ঁংঃঅআইঈউঊঋএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহ়ািীুূৃেৈোৌ্ৎৗড়ঢ়য়০১২৩৪৬৭৮৯‌‍–—―’”−


In [2]:
alamgir_text = text
len(alamgir_text)

660437

In [7]:
adrita_text = text
len(adrita_text)

793163

## Creating `characters` for COQUI-TTS

In [9]:
text = alamgir_text + adrita_text
chars = set(text)
print(f"Characters in corpus: {chars}")

Characters in corpus: {'ক', 'ড়', '৪', 'ফ', '৫', ' ', ')', 't', 'ৎ', '%', 'ঞ', 'ছ', 'য়', 'ভ', 'শ', 'a', 'ূ', '১', 'H', 'R', 'হ', 'ঁ', 'l', ',', 'F', 'ো', '.', 'া', 'র', '"', 'ষ', 'ৗ', 'প', 'h', 'ঠ', '0', '০', 'Y', 'ণ', '(', 'b', 'L', 'ঙ', 'ঐ', 'থ', '৯', 'x', '*', 'ঔ', '৬', 'য', 'k', 'ধ', 'C', '\u200d', 'খ', 'ই', '3', 'ঌ', 'B', '?', 'জ', '[', 'i', 'গ', 'ী', 'ৈ', 'উ', 'c', 'ঝ', 'I', 'ে', '8', 'e', 'ঢ়', 'M', 'ত', 'G', '২', '#', 'ু', '5', '৭', 'S', 'ৃ', '\xa0', '-', 'K', '৩', 'f', 'o', 'q', 'ও', '―', 'y', 'ঃ', 'ং', 'd', 'm', '/', 'T', '়', 'ব', 'স', '6', 'চ', '্', 'A', 'আ', '–', 'n', ';', 'u', '\u200c', '−', 'ন', 'w', 'E', 's', '!', 'r', '°', '—', 'ঈ', 'ি', 'J', '@', '৮', '’', 'ৌ', 'ড', 'অ', 'g', 'ঘ', '”', 'ম', 'D', ']', 'ঋ', ':', 'এ', 'ট', 'N', 'ঢ', 'ঊ', 'P', 'p', '1', 'ল', '।', "'", 'দ'}


In [48]:
chars = "@ABCDEFGHIJKLMNPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
puncs = ";:,.!?—…\"«»“” "

In [9]:
for c in chars:
    #if ord(c) < 128: continue # ASCII
        
    print(f"{c} -> \\u{ord(c):04x}")

0 -> \u0030
1 -> \u0031
2 -> \u0032
3 -> \u0033
4 -> \u0034
5 -> \u0035
6 -> \u0036
7 -> \u0037
8 -> \u0038
9 -> \u0039
@ -> \u0040
A -> \u0041
B -> \u0042
C -> \u0043
D -> \u0044
E -> \u0045
F -> \u0046
G -> \u0047
H -> \u0048
I -> \u0049
J -> \u004a
K -> \u004b
L -> \u004c
M -> \u004d
N -> \u004e
P -> \u0050
Q -> \u0051
R -> \u0052
S -> \u0053
T -> \u0054
U -> \u0055
V -> \u0056
W -> \u0057
X -> \u0058
Y -> \u0059
Z -> \u005a
a -> \u0061
b -> \u0062
c -> \u0063
d -> \u0064
e -> \u0065
f -> \u0066
g -> \u0067
h -> \u0068
i -> \u0069
j -> \u006a
k -> \u006b
l -> \u006c
m -> \u006d
n -> \u006e
o -> \u006f
p -> \u0070
q -> \u0071
r -> \u0072
s -> \u0073
t -> \u0074
u -> \u0075
v -> \u0076
w -> \u0077
x -> \u0078
y -> \u0079
z -> \u007a


In [49]:
puncs += r"\u" + f"{ord('।'):04x}"
print(puncs)

;:,.!?—…"«»“” \u0964


In [28]:
int("9e0", 16)

2528

In [44]:
i = [x for x in range(int("981",16), int(""))] + [y for y in range(5, 10)]
i

[1, 2, 5, 6, 7, 8, 9]

In [50]:
for i in range(int("981",16), int("9e0",16)):
    print(f"\\u{i:04x} -> {chr(i)}")
    chars += f"\\u{i:04x}"

\u0981 -> ঁ
\u0982 -> ং
\u0983 -> ঃ
\u0984 -> ঄
\u0985 -> অ
\u0986 -> আ
\u0987 -> ই
\u0988 -> ঈ
\u0989 -> উ
\u098a -> ঊ
\u098b -> ঋ
\u098c -> ঌ
\u098d -> ঍
\u098e -> ঎
\u098f -> এ
\u0990 -> ঐ
\u0991 -> ঑
\u0992 -> ঒
\u0993 -> ও
\u0994 -> ঔ
\u0995 -> ক
\u0996 -> খ
\u0997 -> গ
\u0998 -> ঘ
\u0999 -> ঙ
\u099a -> চ
\u099b -> ছ
\u099c -> জ
\u099d -> ঝ
\u099e -> ঞ
\u099f -> ট
\u09a0 -> ঠ
\u09a1 -> ড
\u09a2 -> ঢ
\u09a3 -> ণ
\u09a4 -> ত
\u09a5 -> থ
\u09a6 -> দ
\u09a7 -> ধ
\u09a8 -> ন
\u09a9 -> ঩
\u09aa -> প
\u09ab -> ফ
\u09ac -> ব
\u09ad -> ভ
\u09ae -> ম
\u09af -> য
\u09b0 -> র
\u09b1 -> ঱
\u09b2 -> ল
\u09b3 -> ঳
\u09b4 -> ঴
\u09b5 -> ঵
\u09b6 -> শ
\u09b7 -> ষ
\u09b8 -> স
\u09b9 -> হ
\u09ba -> ঺
\u09bb -> ঻
\u09bc -> ়
\u09bd -> ঽ
\u09be -> া
\u09bf -> ি
\u09c0 -> ী
\u09c1 -> ু
\u09c2 -> ূ
\u09c3 -> ৃ
\u09c4 -> ৄ
\u09c5 -> ৅
\u09c6 -> ৆
\u09c7 -> ে
\u09c8 -> ৈ
\u09c9 -> ৉
\u09ca -> ৊
\u09cb -> ো
\u09cc -> ৌ
\u09cd -> ্
\u09ce -> ৎ
\u09cf -> ৏
\u09d0 -> ৐
\u09d1 -> ৑
\u09d2 -> ৒
\u09d3 -> ৓
\u09

In [51]:
print(chars)

@ABCDEFGHIJKLMNPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz\u0981\u0982\u0983\u0984\u0985\u0986\u0987\u0988\u0989\u098a\u098b\u098c\u098d\u098e\u098f\u0990\u0991\u0992\u0993\u0994\u0995\u0996\u0997\u0998\u0999\u099a\u099b\u099c\u099d\u099e\u099f\u09a0\u09a1\u09a2\u09a3\u09a4\u09a5\u09a6\u09a7\u09a8\u09a9\u09aa\u09ab\u09ac\u09ad\u09ae\u09af\u09b0\u09b1\u09b2\u09b3\u09b4\u09b5\u09b6\u09b7\u09b8\u09b9\u09ba\u09bb\u09bc\u09bd\u09be\u09bf\u09c0\u09c1\u09c2\u09c3\u09c4\u09c5\u09c6\u09c7\u09c8\u09c9\u09ca\u09cb\u09cc\u09cd\u09ce\u09cf\u09d0\u09d1\u09d2\u09d3\u09d4\u09d5\u09d6\u09d7\u09d8\u09d9\u09da\u09db\u09dc\u09dd\u09de\u09df


In [54]:
f"{8217:04x}"

'2019'

In [52]:
ord('’')

8217